### 导入数据，进行初步的EDA。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



df = pd.read_csv('../data/summerOly_athletes.csv')

当前工作目录: e:\python\2025-MCM-ICM\code


### 进行特征筛选与特征降维
- 首先分析变量与目标变量之间的相关性，然后分析变量与目标变量的互信息，综合考虑两者的结果，进行特征选择。
- 观察看有没有必要进行特征降维，如果有，可以考虑使用PCA等方法进行特征降维。

In [ ]:
# 111

### 接下来完成第一步也就是零膨胀模型的构建
- 首先判断历史数据的平稳性，使用Augmented Dickey-Fuller检验验证奖牌数时间序列的平稳性。
- 然后为第一部分（逻辑回归）选择合适的特征，使用逻辑回归模型预测从来没有获得过奖牌的国家在下一届奥运会上获得奖牌的概率。
- 对于第二部分（计数部分），如果数据不平稳就选择负二项分布，如果数据平稳就选择泊松分布。
- 最后，将两部分的结果相乘，得到最终的预测结果。


In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP


def check_stationarity(df):
    for col in df.columns:
        # ADF test
        result = adfuller(df[col])
        print(f'ADF Statistic for {col}: {result[0]}')
        print(f'p-value: {result[1]}')
        if result[1] > 0.05:
            return False
    return True

## 开始训练零膨胀负二项模型

features = {
    'weight': 'weight',
    'height': 'height',
}

target = 'medals_gt_zero'

# 划分训练集和测试集
from sklearn.model_selection import train_test_split
X = df[features.keys()]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
formular = f'{target} ~ {" + ".join(features.values())}'

# 训练模型
model = ZeroInflatedNegativeBinomialP.from_formula(formular, data=X_train,exog_infl=~1)
zinb_results = model.fit(maxiter=1000)

## 预测
y_pred = zinb_results.predict(X_test,which='prob')

#计算ROC曲线
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)

#寻找最佳阈值
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
plt.scatter(fpr[optimal_idx], tpr[optimal_idx], color='red', label='Best Threshold')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')



#预测下一届奥运会哪些国家第一次获得奖牌

next_year = 2028
prediction = zinb_results.predict(df[features.keys()])
df['prediction'] = prediction






        

### 第二阶段使用混合预测模型
- 首先使用ARIMAX进行时间序预测，结合历史数据和外部因素，预测下一届奥运会的奖牌数。
- 然后结合Xgboost进行残差预测，得到最终的预测结果。

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from sklearn.preprocessing import StandardScaler

## 平稳性处理模块
def make_stationary(series):
    d = 0 
    while adfuller(series)[1] > 0.05:
        series = series.diff().dropna()
        d += 1
    return series, d

#从外部变量预处理模块
def prepare